<a href="https://colab.research.google.com/github/JitsukawaNoAniki/DSWA-09/blob/main/markov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pomegranate
!pip install networkx
import networkx as nx
!pip install numpy
import numpy as np
import csv
from pomegranate import *
from networkx import *
from numpy import *
!pip install pyvis
from pyvis.network import Network
import os

fonte = []

dados = []

chance = fonte

simple = True

#Para defesas, remover laterais; para ataque remover meias.

#defesas = ["GOL", "ZAG E", "ZAG D", "LAT E", "LAT D"]
#ataques = ["VOL", "MEI E", "MEI D", "ATA E", "ATA D", "ATA M"]

defesas = ["GOL", "ZAG E", "ZAG D"]
ataques = ["VOL", "ATA D", "ATA M"]

pcoords = {"GOL":[-600, 0],
           "ZAG E":[-300, -90],
           "ZAG D":[-300, 90],
           "LAT E":[-250, -270],
           "LAT D":[-250, 270],
           "VOL":[-60, 0],
           "MEI E":[60, -225],
           "MEI D":[60, 225],
           "ATA E":[300, -120],
           "ATA D":[300, 120],
           "ATA M":[300, 0],
           "defesas":[0, 0],
           "finalizações total":[600, 0],
           "finalizações certas":[600, 0],
           "gols": [600, 0]}

def Zerado(tamanho):
  a=0
  while a<(tamanho-2):
    fonte.append([0])
    b=0
    while b<(tamanho-3):
      fonte[a].append(0)
      b+=1
    a+=1
  dados.append([""])
  dados.append([""])
  b=0
  while b<(tamanho-7):
      dados[0].append("")
      dados[1].append("")
      b+=1

def Read(table):
  exp = ""
  a=0
  while a<len(table):
    if a<10: 
      exp = ":  "
    else:
      exp = ": "
    print(str(a) + exp + str(table[a]))
    a+=1

def Posicionar(graph, player):
  nx.set_node_attributes(graph, {player:int(pcoords[playerpos[player]][0])}, 'x')
  nx.set_node_attributes(graph, {player:int(pcoords[playerpos[player]][1])}, 'y')

def ColorirCaminho(graph, paths, color):
  for caminho in paths:
    e = 0
    while e<len(caminho)-1:
      print(str(caminho[e]) + " to " + str(caminho[e+1]) + " em " + color)
      nx.set_edge_attributes(graph, {(caminho[e], caminho[e+1]) : color}, 'color')
      e+=1

def Build(g, a):
  a.from_nx(g)
  a.barnes_hut(gravity=0, central_gravity=0, spring_length=0, spring_strength=0, damping=0, overlap=0)
  a.toggle_physics(False)

def Limpar(graph, paths):
  a=0
  while a<limite:
      b=0
      while b<limite:
        if grafo[players[a]][players[b]] not in paths:
          grafo.remove_edge(players[a], players[b])
        b+=1
      a+=1

#nx.set_node_attributes(grafo, {'A':-600}, 'x')

limite = 0

#Zerado(limite)
#Read(fonte)

a = 0
b = 0

caminho = '/content/footstats_data'

#print(os.listdir(caminho))

first = True

for tabela in os.listdir(caminho):
  if tabela.endswith('.csv'):
    with open(caminho + "/" + tabela, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
          if first:
            players = list(row.keys())
            Read(players)
            limite = len(players)
            players.pop(limite-1)
            players.pop(limite-2)
            Zerado(limite)
            first = False
          if b<limite-6 or b==limite-3:
            a=0
            while a<limite-6:
              fonte[b][a]+=int(row[players[a]])
              a+=1
          elif b<limite-3 and b<limite-1:
            a=0
            while a<limite-6:
              fonte[a][b]+=int(row[players[a]])
              a+=1
          else:
            a=0
            while a<limite-6:
              dados[(b-limite)+1][a]=row[players[a]]
              a+=1
          b+=1

playerpos = {}

a=0
while a<len(dados[0]):
  playerpos[dados[1][a]] = dados[0][a]
  a+=1

#Read(players)
#Read(fonte)
#Read(dados)

limite-=2

c = 0
s = 0

print(len(dados[0]))

#Matriz de probabilidade do passe
a = 0
while a<limite:
    b=0
    while b<limite:
        c=0
        s=0
        while c<limite:
            s+=fonte[a][c]
            c+=1
        if s == 0:
          chance[a][b]=0
        if s>0:
          chance[a][b]=((fonte[a][b])/s)
        b+=1
    a+=1

if simple:
  a = 0
  while a<limite-4:
    b=0
    while b<limite-4:
      if playerpos[players[a]] in defesas:
        chance[a][limite-2]=0
        chance[a][limite-3]=0
        chance[a][limite-4]=0
      if playerpos[players[a]] in ataques:
        chance[limite-1][a]=0
      b+=1
    a+=1

Read(chance)

#contrução da matriz de transição
matriz = []

a=0
while a<limite:
    b=0
    while b<limite:
      matriz.append([players[a], players[b], chance[a][b]])
      b+=1
    a+=1

#Read(matriz)

distro = {}
a=0
while a<limite:
  distro[players[a]] = 1
  a+=1

vv = DiscreteDistribution(distro)
vy = ConditionalProbabilityTable(matriz, [vv])
vm = MarkovChain([vv, vy])

chanceprob = []

a=0
while a<limite:
    b=0
    while b<limite:
      chanceprob.append([players[a], players[b], 1-(np.exp(vm.log_probability([players[a], players[b]])))])
      b+=1
    a+=1

#Read(chanceprob)

grafo = nx.DiGraph()

grafo.add_weighted_edges_from(chanceprob)

nx.set_node_attributes(grafo, {'defesas':int(pcoords['defesas'][0])}, 'x')
nx.set_node_attributes(grafo, {'defesas':int(pcoords['defesas'][1])}, 'y')
nx.set_node_attributes(grafo, {'finalizações total':int(pcoords['finalizações total'][0])}, 'x')
nx.set_node_attributes(grafo, {'finalizações total':int(pcoords['finalizações total'][1])}, 'y')
nx.set_node_attributes(grafo, {'finalizações certas':int(pcoords['finalizações certas'][0])}, 'x')
nx.set_node_attributes(grafo, {'finalizações certas':int(pcoords['finalizações certas'][1])}, 'y')
nx.set_node_attributes(grafo, {'gols':int(pcoords['gols'][0])}, 'x')
nx.set_node_attributes(grafo, {'gols':int(pcoords['gols'][1])}, 'y')

print(grafo.nodes)

a=0
while a<(limite-4):
  Posicionar(grafo, players[a])
  a+=1

grafo.add_node('IE', x=-600, y=450)
grafo.add_node('ID', x=600, y=450)
grafo.add_node('SE', x=-600, y=-450)
grafo.add_node('SD', x=600, y=-450)

#nx.set_node_attributes(grafo, {'IE' : True,'ID' : True,'SE' : True,'SD' : True}, 'hidden')

nx.set_node_attributes(grafo, True, 'fixed')

nx.set_node_attributes(grafo, 'blue', 'color')

#nx.set_node_attributes(grafo, 50, 'size')
nx.set_edge_attributes(grafo, 5, 'width')

a=0
while a<limite:
    b=0
    while b<limite:
      if grafo[players[a]][players[b]]['weight'] == 1.0:
        grafo.remove_edge(players[a], players[b])
      b+=1
    a+=1

#nx.draw(grafo, with_labels=True)

res1 = nx.DiGraph()
res1 = grafo
res1.remove_node('finalizações certas')
res1.remove_node('gols')

res2 = nx.DiGraph()
res2 = grafo
res2.remove_node('finalizações total')
res2.remove_node('gols')

res3 = nx.DiGraph()
res3 = grafo
res3.remove_node('finalizações total')
res3.remove_node('finalizações certas')

shortest_path_final_total = ([p for p in nx.all_shortest_paths(res1, source=players[limite-1], target=players[limite-4], weight='weight')])
ColorirCaminho(res1, shortest_path_final_total, 'red')
Limpar(res1, shortest_path_final_total)

shortest_path_final_certa = ([p for p in nx.all_shortest_paths(res2, source=players[limite-1], target=players[limite-3], weight='weight')])
ColorirCaminho(res2, shortest_path_final_certa, 'yellow')
Limpar(res2, shortest_path_final_certa)

shortest_path_gols = ([p for p in nx.all_shortest_paths(res3, source=players[limite-1], target=players[limite-2], weight='weight')])
ColorirCaminho(res3, shortest_path_gols, 'green')
Limpar(res3, shortest_path_gols)

#print(dijkstra_path(grafo, 'defesas', 'finalizações', weight='weight'))

gfinalTotal = Network("624px","960px")
gfinalCertas = Network("624px","960px")
ggols = Network("624px","960px")

Build(res1, gfinalTotal)
Build(res2, gfinalCertas)
Build(res3, ggols)

gfinalTotal.show('1.html')
gfinalCertas.show('2.html')
ggols.show('3.html')

#print(grafo.edges)

#adiciona o campo no arquivo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.3 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pomegranate: filename=pomegranate-0.14.8-cp37-cp37m-linux_x86_64.whl size=15066299 sha256=8fba0eaf292d63362e07dad0ae0486eb05cee19753b6ad51704d6aa6c4b815fb
  Stored in directory: /root/.cache/pip/wheels/24/68/69/0eaab474ef1d65abedcd47de8a38ab21d221d329954d7edd24
Successfully built pomegranate
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 748 kB 5.2 MB/s 
  Created wheel for pyvis: filename=pyvis-0.3.1-py3

NetworkXError: ignored

background-image: url("https://st.depositphotos.com/2039187/2006/i/600/depositphotos_20060343-stock-photo-soccer-field-or-football-field.jpg"); background-repeat: no-repeat;

background-image: url("https://media.istockphoto.com/id/959817902/pt/vetorial/football-soccer-court-sport-background-line-art-style.jpg?s=612x612&w=0&k=20&c=drr2c8VQjJNezcN5Yu0xiAe-CTASf4pGdKUZGDcgrYk="); background-repeat: no-repeat;

In [ ]:
!pip install pomegranate
!pip install networkx
import networkx as nx
!pip install numpy
import numpy as np
import csv
from pomegranate import *
from networkx import *
from numpy import *
!pip install pyvis
from pyvis.network import Network
import os

fonte = []

dados = []

chance = fonte

simple = True

#Para defesas, remover laterais; para ataque remover meias.

#defesas = ["GOL", "ZAG E", "ZAG D", "LAT E", "LAT D"]
#ataques = ["VOL", "MEI E", "MEI D", "ATA E", "ATA D", "ATA M"]

defesas = ["GOL", "ZAG E", "ZAG D"]
ataques = ["VOL", "ATA D", "ATA M"]

pcoords = {"GOL":[-600, 0],
           "ZAG E":[-300, -90],
           "ZAG D":[-300, 90],
           "LAT E":[-250, -270],
           "LAT D":[-250, 270],
           "VOL":[-60, 0],
           "MEI E":[60, -225],
           "MEI D":[60, 225],
           "ATA E":[300, -120],
           "ATA D":[300, 120],
           "ATA M":[300, 0],
           "defesas":[0, 0],
           "finalizações total":[600, 0],
           "finalizações certas":[600, 0],
           "gols": [600, 0]}

def Zerado(tamanho):
  a=0
  while a<(tamanho-2):
    fonte.append([0])
    b=0
    while b<(tamanho-3):
      fonte[a].append(0)
      b+=1
    a+=1
  dados.append([""])
  dados.append([""])
  b=0
  while b<(tamanho-7):
      dados[0].append("")
      dados[1].append("")
      b+=1

def Read(table):
  exp = ""
  a=0
  while a<len(table):
    if a<10: 
      exp = ":  "
    else:
      exp = ": "
    print(str(a) + exp + str(table[a]))
    a+=1

def Posicionar(graph, player):
  nx.set_node_attributes(graph, {player:int(pcoords[playerpos[player]][0])}, 'x')
  nx.set_node_attributes(graph, {player:int(pcoords[playerpos[player]][1])}, 'y')

def ColorirCaminho(graph, paths, color, clear):
  if clear:
    graph.remove_edges_from(list(graph.edges))
  for caminho in paths:
    e = 0
    while e<len(caminho)-1:
      print(str(caminho[e]) + " to " + str(caminho[e+1]) + " em " + color)
      graph.add_edge(caminho[e], caminho[e+1])
      nx.set_edge_attributes(graph, {(caminho[e], caminho[e+1]) : color}, 'color')
      nx.set_node_attributes(graph, {caminho[e] : color}, 'color')
      e+=1

def Montar(grafo):
  grafo.add_weighted_edges_from(chanceprob)

  nx.set_node_attributes(grafo, {'defesas':int(pcoords['defesas'][0])}, 'x')
  nx.set_node_attributes(grafo, {'defesas':int(pcoords['defesas'][1])}, 'y')
  nx.set_node_attributes(grafo, {'finalizações total':int(pcoords['finalizações total'][0])}, 'x')
  nx.set_node_attributes(grafo, {'finalizações total':int(pcoords['finalizações total'][1])}, 'y')
  nx.set_node_attributes(grafo, {'finalizações certas':int(pcoords['finalizações certas'][0])}, 'x')
  nx.set_node_attributes(grafo, {'finalizações certas':int(pcoords['finalizações certas'][1])}, 'y')
  nx.set_node_attributes(grafo, {'gols':int(pcoords['gols'][0])}, 'x')
  nx.set_node_attributes(grafo, {'gols':int(pcoords['gols'][1])}, 'y')

  print(grafo.nodes)

  a=0
  while a<(limite-4):
    Posicionar(grafo, players[a])
    a+=1

  grafo.add_node('IE', x=-600, y=450)
  grafo.add_node('ID', x=600, y=450)
  grafo.add_node('SE', x=-600, y=-450)
  grafo.add_node('SD', x=600, y=-450)

  #nx.set_node_attributes(grafo, {'IE' : True,'ID' : True,'SE' : True,'SD' : True}, 'hidden')

  nx.set_node_attributes(grafo, True, 'fixed')

  nx.set_node_attributes(grafo, 'black', 'color')

  #nx.set_node_attributes(grafo, 50, 'size')
  nx.set_edge_attributes(grafo, 5, 'width')

  a=0
  while a<limite:
      b=0
      while b<limite:
        if grafo[players[a]][players[b]]['weight'] == 1.0:
          grafo.remove_edge(players[a], players[b])
        b+=1
      a+=1

def Build(g, a):
  a.from_nx(g)
  a.barnes_hut(gravity=0, central_gravity=0, spring_length=0, spring_strength=0, damping=0, overlap=0)
  a.toggle_physics(False)

#nx.set_node_attributes(grafo, {'A':-600}, 'x')

limite = 0

#Zerado(limite)
#Read(fonte)

a = 0
b = 0

caminho = '/content/footstats_data'

#print(os.listdir(caminho))

first = True

for tabela in os.listdir(caminho):
  if tabela.endswith('.csv'):
    with open(caminho + "/" + tabela, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
          if first:
            players = list(row.keys())
            Read(players)
            limite = len(players)
            players.pop(limite-1)
            players.pop(limite-2)
            Zerado(limite)
            first = False
          if b<limite-6 or b==limite-3:
            a=0
            while a<limite-6:
              fonte[b][a]+=int(row[players[a]])
              a+=1
          elif b<limite-3 and b<limite-1:
            a=0
            while a<limite-6:
              fonte[a][b]+=int(row[players[a]])
              a+=1
          else:
            a=0
            while a<limite-6:
              dados[(b-limite)+1][a]=row[players[a]]
              a+=1
          b+=1

playerpos = {}

a=0
while a<len(dados[0]):
  playerpos[dados[1][a]] = dados[0][a]
  a+=1

#Read(players)
#Read(fonte)
#Read(dados)

limite-=2

c = 0
s = 0

print(len(dados[0]))

#Matriz de probabilidade do passe
a = 0
while a<limite:
    b=0
    while b<limite:
        c=0
        s=0
        while c<limite:
            s+=fonte[a][c]
            c+=1
        if s == 0:
          chance[a][b]=0
        if s>0:
          chance[a][b]=((fonte[a][b])/s)
        b+=1
    a+=1

if simple:
  a = 0
  while a<limite-4:
    b=0
    while b<limite-4:
      if playerpos[players[a]] not in defesas:
        chance[limite-1][a]=0
      if playerpos[players[a]] not in ataques:
        chance[a][limite-2]=0
        chance[a][limite-3]=0
        chance[a][limite-4]=0
      b+=1
    a+=1

Read(chance)

#contrução da matriz de transição
matriz = []

a=0
while a<limite:
    b=0
    while b<limite:
      matriz.append([players[a], players[b], chance[a][b]])
      b+=1
    a+=1

#Read(matriz)

distro = {}
a=0
while a<limite:
  distro[players[a]] = 1
  a+=1

vv = DiscreteDistribution(distro)
vy = ConditionalProbabilityTable(matriz, [vv])
vm = MarkovChain([vv, vy])

chanceprob = []

a=0
while a<limite:
    b=0
    while b<limite:
      chanceprob.append([players[a], players[b], 1-(np.exp(vm.log_probability([players[a], players[b]])))])
      b+=1
    a+=1

#Read(chanceprob)

#nx.draw(grafo, with_labels=True)

grafoTodos = nx.DiGraph()
Montar(grafoTodos)

res1 = nx.DiGraph()
Montar(res1)
res1.remove_node('finalizações certas')
res1.remove_node('gols')

res2 = nx.DiGraph()
Montar(res2)
res2.remove_node('finalizações total')
res2.remove_node('gols')

res3 = nx.DiGraph()
Montar(res3)
res3.remove_node('finalizações total')
res3.remove_node('finalizações certas')

shortest_path_final_total = ([p for p in nx.all_shortest_paths(res1, source=players[limite-1], target=players[limite-4], weight='weight')])
ColorirCaminho(res1, shortest_path_final_total, 'red', True)

shortest_path_final_certa = ([p for p in nx.all_shortest_paths(res2, source=players[limite-1], target=players[limite-3], weight='weight')])
ColorirCaminho(res2, shortest_path_final_certa, 'yellow', True)

shortest_path_gols = ([p for p in nx.all_shortest_paths(res3, source=players[limite-1], target=players[limite-2], weight='weight')])
ColorirCaminho(res3, shortest_path_gols, 'blue', True)

#print(dijkstra_path(grafo, 'defesas', 'finalizações', weight='weight'))

gtotal = Network("390px","600px")
gfinalTotal = Network("390px","600px")
gfinalCertas = Network("390px","600px")
ggols = Network("390px","600px")

Build(grafoTodos, gtotal)
Build(res1, gfinalTotal)
Build(res2, gfinalCertas)
Build(res3, ggols)

gtotal.show('0.html')
gfinalTotal.show('1.html')
gfinalCertas.show('2.html')
ggols.show('3.html')

#print(grafo.edges)

#adiciona o campo no arquivo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
0:  Alef Manga
1:  Alex Muralha
2:  Andrey
3:  Egídio
4:  Henrique
5:  Igor Paixão
6:  Léo Gamalho
7:  Luciano Castan
8:  Thonny Anderson
9:  Warley
10: William Farias
11: finalizações total
12: finalizações certas
13: gols
14: defesas
15: nome
16: pos
11
0:  [0.0, 0.0, 0.06818181818181818, 0.14609850581073602, 0.0, 0.08519271080931361, 0.09288077228899605, 0.06804490763572336, 0.18208038608407304, 0.0, 0.30915342656756084, 0, 0, 0, 0.0]
1:  [0.0, 0.0, 0.045454545454545456, 0.04751619870410367, 0.29861189151157735, 0.0, 0.0, 0.9033057954153616, 0.0, 0.0, 0.4357510310542434, 0, 0, 0, 0.0]
2:  [0.10

colocar zaga e goleiro no nó defesa
tratar pelo nome/posição
LOOKUP table
rever posições
por enquanto, BG na mão
body {
  background-image: url("img_tree.png");
  background-repeat: no-repeat;
}